# Data cleanup

In [18]:
import pandas as pd
train = pd.read_csv('data/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [19]:
test = pd.read_csv('data/test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## Remove duplicates

In [20]:
len(train)

7613

In [21]:
train.drop_duplicates(subset=['keyword','location','text','target'], inplace=True)

In [22]:
len(train)

7561

## Remove hashtags from text

In [23]:
train['cleaned_text'] = train['text'].str.replace('#'," ")
test['cleaned_text'] = test['text'].str.replace('#'," ")

In [24]:
sum(train['text'].str.count('#'))

3359

In [25]:
sum(train['cleaned_text'].str.count('#'))

0

## Remove URLs, smileys and mentions (@) from text
Using Tweet Preprocessor: https://pypi.org/project/tweet-preprocessor/

In [26]:
import preprocessor

In [27]:
train['cleaned_text'] = train['cleaned_text'].apply(lambda x: preprocessor.clean(x))

In [28]:
test['cleaned_text'] = test['cleaned_text'].apply(lambda x: preprocessor.clean(x))

In [29]:
train['cleaned_text'].iloc[200]

'Twelve feared killed in Pakistani air ambulance helicopter crash worldnews'

# Keyword cleanup

In [35]:
train.loc[~train['keyword'].isnull(), 'keyword'].unique()

array(['ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'derailed

In [37]:
train['keyword'] = train['keyword'].str.replace('%20', " ")

In [38]:
train.loc[~train['keyword'].isnull(), 'keyword'].unique()

array(['ablaze', 'accident', 'aftershock', 'airplane accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown up', 'body bag', 'body bagging', 'body bags',
       'bomb', 'bombed', 'bombing', 'bridge collapse',
       'buildings burning', 'buildings on fire', 'burned', 'burning',
       'burning buildings', 'bush fires', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'chemical emergency', 'cliff fall',
       'collapse', 'collapsed', 'collide', 'collided', 'collision',
       'crash', 'crashed', 'crush', 'crushed', 'curfew', 'cyclone',
       'damage', 'danger', 'dead', 'death', 'deaths', 'debris', 'deluge',
       'deluged', 'demolish', 'demolished', 'demolition', 'derail',
       'derailed', 'derailment', 'desolate',

# Saving data

In [39]:
test.to_csv('test_cleaned.csv', index=False)

In [40]:
train.to_csv('train_cleaned.csv', index=False)